In [ ]:
input_fname = './Onegin/onegin.txt'

In [ ]:
START_CHAR  = '\b'
END_CHAR = '\t'
PADDING_CHAR = '\a'
chars = set([START_CHAR,'\n',END_CHAR])
with open(input_fname,'r') as f:
    for line in f:
        chars.update(list(line.strip().lower()))
char_indices = {c: i for i,c in enumerate(sorted(list(chars)))}
char_indices[PADDING_CHAR] = 0
indices_to_chars = {i:c for c,i  in char_indices.items()}
num_chars = len(chars)

In [ ]:
import numpy as np

In [ ]:
def get_one(i, sz):
    res = np.zeros(sz)
    res[i] = 1
    return res

char_vectors = {
    c:(np.zeros(num_chars) if c==PADDING_CAHR else get_one(v,num_chars))
    for c,v in char_indices.items()
}

In [ ]:
sentence_end_markers = set('.?!')
sentences = []
current_sentence = ''
with open(input_fname, 'r') as f:
    for line in f:
        s = line.strip().lower()
        if len(s) > 0:
            current_sentence+=s +'\n'
        if len(s) == 0 or s[-1] in sentence_end_markers:
            current_sentence = current_sentence.strip()
            if len(current_sentence) > 10:
                sentences.append(current_sentence)
            current_sentence = ''

In [ ]:
def get_matrix(sentences):
    max_sentence_len = np.max([len(x) for x in sentences])
    X = np.zeros((len(sentences), max_sentence_len, len(chars)),dtype=np.bool)
    y = np.zeros((len(sentences), max_sentence_len, len(chars)),dtype=np.bool)
    for i, sentence in enumerate(sentences):
        char_seq = (START_CHAR + sentence + END_CHAR).ljust( max_sentence_len+1,
                                                           PADDING_CHAR
                                                           )
        for t in range(max_sentence_len):
            X[i,t,:] = char_vectors[char_seq[t]]
            y[i,t,:] = char_vectors[char_seq[t+1]]
    return X,y

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, TimeDistributed, Activation
from keras.optimizers import Adam

In [ ]:


model = Sequential()
model.add(LSTM(output_dim=128, 
               activation = 'tanh',
               return_sequences=True,input_dim = num_chars))

model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_dim=num_chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer = Adam(clipnorm=1.),
             metrics = ['accuracy'])


In [ ]:
test_indices = np.random.choice(range(len(sentences)), int(len(sentences)*0.05))
sentences_train = [
    sentences[x] for x in set(range(len(sentences))) - set(test_indices)
]
sentences_test = [sentences[x] for x in test_indices]
X_test, y_test = get_matrix(sentences_test)
batch_size  = 16
def generate_batch():
    while True:
        for i in range( int(len(sentences_train)/batch_size) ):
            sentences_batch = sentences_train[i*batch_size:(i+1)*batch_size]
            yield get_matrix(sentences_batch)

In [ ]:
from keras.callbacks import Callback

import os 
output_file = 'out'
class CharSampler(Callback):
    
    def __init__(self,char_vectors, model):
        self.char_vectors = char_vectors
        self.model = model
        
    def on_train_begin(self,logs={}):
        self.epoch = 0
        if os.path.isfile(output_file):
            os.remove(output_file)
    
    def sample(self,preds,temperatue=1.0):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds)/temperatue
        exp_preds = np.exp(preds)
        preds = exp_preds/np.sum(exp_preds)
        probas = np.random.multinomial(1,preds,1)
        return np.argmax(probas)
    
    def sample_one(self, T):
        result = START_CHAR
        while len(result) < 500:
            Xsampled = np.zeros((1,len(result), num_chars))
            for t,c in enumerate(result):
                Xsampled[0,t,:] = self.char_vectors[c]
            ysampled= self.model.predict(Xsampled, batch_size=1)[0,:]
            yv = ysampled[len(result)-1,:]
            selected_char = indices_to_chars[self.sample(yv,T)]
            if selected_char == END_CHAR:
                break
            result  = result + selected_char
        return result
    
    def one_epoch_end(self,batch,logs={}):
        self.epoch = self.epoch+1
        if self.epoch %50==0:
            print(f'\n Epoch {self.epoch}  text sampling:')
            with open(output_file, 'a') as outf:
                outf.write(f'\n====Epoch {self.epoch} ====')
                for T in [0.3,0.5,0.7,0.9,1.1]:
                    print(f'\tsampling, T = {T}')
                    for _ in range(5):
                        self.model.reset_states()
                        res = self.sample_one(T)
                        outf.write(f'\nT = {T} = {res}')
                        

In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger

model_fname = 'LSTM'

cb_sampler = CharSampler(char_vectors, model)
cb_logger = CSVLogger('./' + model_fname + '.log')

In [ ]:
model.fit_generator(generate_batch(),
                   int(len(sentences)/batch_size)*batch_size,
                   nb_epoch=1,verbose=True,validation_data=(X_test,y_test),
                   callbacks=[cb_logger,cb_sampler]
                   )


In [ ]:
from keras.layers import merge, Input

vec = Input(shape=[None, num_chars])
l1 = LSTM(output_dim=128, activation='tanh', return_sequences=True)(vec)
l1_d = Dropout(0.2)(l1)

input2 = merge([vec,l1_d],mode='concat')
l2 = LSTM(output_dim=128, activation='tanh', return_sequences=True)(input2)
l2_d = Dropout(0.2)(l2)

input3 = merge([vec,l2_d],mode='concat')
l3 = LSTM(output_dim=128, activation='tanh', return_sequences=True)(input3)
l3_d = Dropout(0.2)(l2)

input_d = merge([l1_d,l2_d,l3_d], mode='concat')

dense3 = TimeDistributed(
    Dense(output_dim=num_chars)
)(input_d)
output_res = Activation('softmax')(dense3)

model = Model(input=vec, output= output_res)
model.compile(loss='categorical_crossentropy',
             optimizer = Adam(clipnorm=1.),
             metrics = ['accuracy'])
model.fit_generator(generate_batch(),
                   int(len(sentences)/batch_size)*batch_size,
                   nb_epoch=1,verbose=True,validation_data=(X_test,y_test),
                   callbacks=[cb_logger,cb_sampler]
                   )
